In [ ]:
#Import and plot Sandy
#%matplotlib inline
from climada.hazard import TCTracks
import numpy
import numpy as np

import warnings
warnings.filterwarnings('ignore')
np.warnings = warnings


In [ ]:
tr_sandy = TCTracks.from_ibtracs_netcdf(provider='usa', storm_id='2012296N14283') # Sandy 2012
ax = tr_sandy.plot();
ax.set_title('SANDY') # set title

In [ ]:
#Generate probablistic tracks.
tr_sandy.equal_timestep()
tr_sandy.calc_perturbed_trajectories(nb_synth_tracks=20) #50 tracks takes ~10 seconds.
tr_sandy.plot();

In [ ]:
#tr_sandy.plot_intensity('2012296N14283_gen1');

In [ ]:
#Importing LitPop Exposure
from climada.entity import LitPop

exp = LitPop.from_countries(['USA'], fin_mode = 'gdp', res_arcsec = 3000) # you can provide either single countries or a list of countries


In [ ]:
exp.plot_scatter();

In [ ]:
import numpy as np
from climada.entity import ImpactFuncSet, ImpfTropCyclone, Exposures
from climada.entity.measures import Measure, MeasureSet
from climada.hazard import Hazard
from climada.engine import ImpactCalc

# define measure
meas = Measure(
    name='Wetlands', # name of measure
    haz_type='TC', # hazard the measure is combating, TC is Tropical Cyclone
    color_rgb=np.array([1, 1, 1]),
    cost=500000000, # cost of measure implementation 
    mdd_impact=(1, 0), # mean damage degree
    paa_impact=(1, 0), # percentage of assets affected
    hazard_inten_imp=(1, -25), # reduces intensity by 25%
)
    
meas.check()

#Make MeasureSet for Entity integration
meas_set = MeasureSet()
meas_set.append(meas)
meas_set.check()

#Imp Funcs
# https://en.wikipedia.org/wiki/Hurricane_Manuel
impf_tc = ImpfTropCyclone.from_emanuel_usa() # This represents some study they did with Emmanuel
impf_all = ImpactFuncSet([impf_tc])

#Hazard: tc_sandy
from climada.hazard import Centroids, TropCyclone

# construct centroids
# NY state
#min_lat, max_lat, min_lon, max_lon = 38, 48, -80, -70

min_lat, max_lat, min_lon, max_lon = 42.6526-0.25, 42.6526+0.25, -73.7562-0.25, -73.7562+0.25

cent = Centroids.from_pnt_bounds((min_lon, min_lat, max_lon, max_lat), res=0.025)
cent.check()
cent.plot();

#Exposure: exp


In [ ]:
# construct tropical cyclones
tc_sandy = TropCyclone.from_tracks(tr_sandy, centroids=cent) # Pass in the centroids from previous cell
tc_sandy.check()


In [ ]:
import matplotlib.pylab as plt

In [ ]:
'''
plt.figure(figsize=(10,10))
for i in range(1,10):
    name = f'2012296N14283_gen{i:d}'
    #ax = plt.subplot(5,5,i+1)
    tc_sandy.plot_intensity(name, figsize=(6,6));  # SANDY's synthetic track
'''

In [ ]:

tc_sandy.plot_intensity('2012296N14283');  # SANDY
tc_sandy.plot_intensity('2012296N14283_gen16');  # SANDY's synthetic track
#gen 8, 16, 

In [ ]:
# impf_all is what we got from the Manuel study
# tc_sandy is our TropicalCyclone derived from Sandy
# exp is the exposure estimated from LitPop

# meas is the Wetlands measured we creatd

new_exp, new_impfs, new_haz = meas.apply(exp, impf_all, tc_sandy)

# if you look at the maximum intensity per centroid: new_haz does not contain the event with smaller impact (the most frequent)
new_haz.plot_intensity(0);


In [ ]:

# you might also compute the exceedance frequency curve of both hazard
imp = ImpactCalc(exp, impf_all, tc_sandy).impact()
ax = imp.calc_freq_curve().plot(label='original');

# With wetlands applied
new_imp = ImpactCalc(new_exp, new_impfs, new_haz).impact()
new_imp.calc_freq_curve().plot(axis=ax, label='measure');
#Impact is in Hundreds of Billions in USD

plt.yscale('log')

plt.legend()

In [ ]:
# impact functions
impf_all.plot();

# new impact functions
axes = new_impfs.plot();
axes.set_title('TC: Modified impact function')

In [ ]:
#Sandy in 50 years
#future_year = 2050

# Bellis edit, 2050 gave me an error. Maybe the files I downloaded?
future_year = 2060

In [ ]:
from climada.util.api_client import Client

client = Client()

haz_present = client.get_hazard('tropical_cyclone', 
                                properties={'country_name': 'United States', 
                                            'climate_scenario': 'historical',
                                            'nb_synth_tracks':'10'})

haz_future = client.get_hazard('tropical_cyclone', 
                                properties={'country_name': 'United States', 
                                            'climate_scenario': 'rcp60',
                                            'ref_year': str(future_year),
                                            'nb_synth_tracks':'10'})


In [ ]:
#Plot present/future
#haz_present.plot_rp_intensity(return_periods=(50,), smooth=False, vmin=32, vmax=50)
#haz_future.plot_rp_intensity(return_periods=(50,), smooth=False, vmin=32, vmax=50)

In [ ]:
#Discount Rates
from climada.entity import DiscRates

# define discount rates
years = np.arange(2000, 2100)
rates = np.ones(years.size) * 0.014
rates[51:55] = 0.025
rates[95:120] = 0.035
disc = DiscRates(years=years, rates=rates)
disc.plot()

# Compute net present value between present year and future year.
ini_year = 2019
#end_year = 2050
end_ear = future_year # Bellis edits
val_years = np.zeros(end_year-ini_year+1)
val_years[0] = 100000000 # initial investment
val_years[10:] = 75000 # maintenance from 10th year
npv = disc.net_present_value(ini_year, end_year, val_years)
print('net present value: {:.5e}'.format(npv))

In [ ]:
#Cost-Benefit Analysis
#Create future hazard (Client or other method, include rcp level)

#Create Entity combining elements.
from climada.entity import Entity

ent = Entity(
    exposures=exp,
    disc_rates=disc,
    impact_func_set=impf_all,
    measure_set=meas_set
)

from climada.engine import CostBenefit
from climada.engine.cost_benefit import risk_aai_agg

#Create the cost-benefit analysis
costben_measures_only = CostBenefit()
costben_measures_only.calc(tc_sandy, ent, haz_future=None, ent_future=None,
                           future_year=None, risk_func=risk_aai_agg, imp_time_depen=None, save_imp=True)